In [26]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
#from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import sequence, text
from keras.utils import pad_sequences
import tensorflow as tf

In [27]:
dt = pd.read_csv('/Users/rishitkapoor/Documents/Python_ML/Generative Text/dataset.csv')
dt.head()

,idx,disease,Symptom,reason,TestsAndProcedures,commonMedications
0,0,Panic disorder,"['Anxiety and nervousness', 'Depression', 'Sho...",Panic disorder is an anxiety disorder characte...,"['Psychotherapy', 'Mental health counseling', ...","['Lorazepam', 'Alprazolam (Xanax)', 'Clonazepa..."
1,1,Vocal cord polyp,"['Hoarse voice', 'Sore throat', 'Difficulty sp...","beclomethasone nasal product,","['Tracheoscopy and laryngoscopy with biopsy', ...","['Esomeprazole (Nexium)', 'Beclomethasone Nasa..."
2,2,Turner syndrome,"['Groin mass', 'Leg pain', 'Hip pain', 'Suprap...",Turner syndrome or Ullrich\xe2\x80\x93Turner s...,"['Complete physical skin exam performed (ML)',...","['Somatropin', 'Sulfamethoxazole (Bactrim)', '..."
3,3,Cryptorchidism,"['Symptoms of the scrotum and testes', 'Swelli...",Cryptorchidism (derived from the Greek \xce\xb...,"['Complete physical skin exam performed (ML)',...","['Haemophilus B Conjugate Vaccine (Obsolete)',..."
4,4,Poisoning due to ethylene glycol,"['Abusing alcohol', 'Fainting', 'Hostile behav...","thiamine,","['Intravenous fluid replacement', 'Hematologic...","['Lorazepam', 'Thiamine', 'Naloxone (Suboxone)..."


In [28]:
dt = dt.drop(['idx'],axis=1)

In [11]:
# General defination of preprocessing for test data
import unicodedata
import string
import textblob
import nltk
import contractions
import re

# remove some stopwords to capture negation in n-grams if possible
stop_words = nltk.corpus.stopwords.words('english')
stop_words.remove('no')
stop_words.remove('not')
stop_words.remove('but')

# load up a simple porter stemmer - nothing fancy
ps = nltk.porter.PorterStemmer()

class Preprocess():
    def remove_accented_chars(text):
        a = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        return a

    def simple_text_preprocessor(document): 
        # lower case
        document = str(document).lower()
        
        # expand contractions
        document = contractions.fix(document)
        
        # remove unnecessary characters
        document = re.sub(r'[^a-zA-Z]',r' ', document)
        document = re.sub(r'nbsp', r'', document)
        document = re.sub(' +', ' ', document)
        
        # simple porter stemming
        document = ' '.join([ps.stem(word) for word in document.split()])
        
        # stopwords removal
        document = ' '.join([word for word in document.split() if word not in stop_words])
        
        return document

In [29]:
dt.columns

Index(['disease', 'Symptom', 'reason', 'TestsAndProcedures',
       'commonMedications'],
      dtype='object')

In [25]:
np.array(dt[['Symptom']].iloc[:,0])

array(["['Anxiety and nervousness', 'Depression', 'Shortness of breath', 'Depressive or psychotic symptoms', 'Sharp chest pain', 'Dizziness', 'Insomnia', 'Abnormal involuntary movements', 'Chest tightness', 'Palpitations', 'Irregular heartbeat', 'Breathing fast']",
       "['Hoarse voice', 'Sore throat', 'Difficulty speaking', 'Cough', 'Nasal congestion', 'Throat swelling', 'Diminished hearing', 'Lump in throat', 'Throat feels tight', 'Difficulty in swallowing', 'Skin swelling', 'Retention of urine']",
       "['Groin mass', 'Leg pain', 'Hip pain', 'Suprapubic pain', 'Blood in stool', 'Lack of growth', 'Diminished hearing', 'Depression', 'Emotional symptoms', 'Elbow weakness', 'Back weakness', 'Pus in sputum']",
       "['Symptoms of the scrotum and testes', 'Swelling of scrotum', 'Pain in testicles', 'Flatulence', 'Pus draining from ear', 'Jaundice', 'Mass in scrotum', 'Lack of growth', 'White discharge from eye', 'Irritable infant']",
       "['Abusing alcohol', 'Fainting', 'Hostile 

In [6]:
a1 = np.array(dt[['Symptom']].iloc[:,0])
a2 = np.array(dt[['reason']].iloc[:,0])
a3 = np.array(dt[['TestsAndProcedures']].iloc[:,0])
a4 = np.array(dt[['commonMedications']].iloc[:,0])
li1 =[]
li2 =[]
li3 =[]
li4= []
for i in a1:
    li1.append(Preprocess.simple_text_preprocessor(i))
dt['Symptom']=[x for x in li1]

for i in a2:
    li2.append(Preprocess.simple_text_preprocessor(i))
dt['reason']=[x for x in li2]

for i in a3:
    li3.append(Preprocess.simple_text_preprocessor(i))
dt['TestsAndProcedures']=[x for x in li3]

for i in a4:
    li4.append(Preprocess.simple_text_preprocessor(i))
dt['commonMedications']=[x for x in li4]

In [7]:
dt.head()

,disease,Symptom,reason,TestsAndProcedures,commonMedications
0,Panic disorder,anxieti nervous depress short breath depress p...,panic disord anxieti disord character recur se...,psychotherapi mental health counsel electrocar...,lorazepam alprazolam xanax clonazepam paroxeti...
1,Vocal cord polyp,hoars voic sore throat difficulti speak cough ...,beclomethason nasal product,tracheoscopi laryngoscopi biopsi occup therapi...,esomeprazol nexium beclomethason nasal product...
2,Turner syndrome,groin mass leg pain hip pain suprapub pain blo...,turner syndrom ullrich xe x x turner syndrom a...,complet physic skin exam perform ml ultrasonog...,somatropin sulfamethoxazol bactrim pimecrolimu...
3,Cryptorchidism,symptom scrotum test swell scrotum pain testic...,cryptorchid deriv greek xce xba xcf x xcf x xc...,complet physic skin exam perform ml ultrasonog...,haemophilu b conjug vaccin obsolet hepat b vac...
4,Poisoning due to ethylene glycol,abus alcohol faint hostil behavior drug abus d...,thiamin,intraven fluid replac hematolog test blood tes...,lorazepam thiamin naloxon suboxon charcoal fol...


In [17]:
numbers = (1, 2, 3, 4)
result = map(lambda x: x + x, numbers)

l = ['sat', 'bat', 'cat', 'mat']
# map() can listify the list of strings individually
test = list(map(list, l))
print(test)

[['s', 'a', 't'], ['b', 'a', 't'], ['c', 'a', 't'], ['m', 'a', 't']]


In [30]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(dt["Symptom"].append(dt["commonMedications"]))

/var/folders/qr/06840hdx0_n24x3_vtrp47rm0000gn/T/ipykernel_6202/1344308809.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tokenizer.fit_on_texts(dt["Symptom"].append(dt["commonMedications"]))


In [33]:
# Define the input sequence length
maxlen = 100

# Encode the symptom and medications columns
X_symptom = tokenizer.texts_to_sequences(dt["Symptom"])
X_medications = tokenizer.texts_to_sequences(dt["commonMedications"])

# Pad the encoded sequences to a fixed length
X_symptom = tf.keras.preprocessing.sequence.pad_sequences(X_symptom, maxlen=maxlen)
X_medications = tf.keras.preprocessing.sequence.pad_sequences(X_medications, maxlen=maxlen)

# Define the model
input_symptom = tf.keras.layers.Input(shape=(maxlen,))
input_medications = tf.keras.layers.Input(shape=(maxlen,))
x = tf.keras.layers.Concatenate()([input_symptom, input_medications])
x = tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128)(x)
x = tf.keras.layers.LSTM(128)(x)
output = tf.keras.layers.Dense(len(set(dt["reason"])), activation='softmax')(x)
model = tf.keras.Model(inputs=[input_symptom, input_medications], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
Y = pd.get_dummies(dt["reason"]).values
model.fit([X_symptom, X_medications], Y, epochs=50, batch_size=32)

# Generate text using the trained model
def generate_text(model, input_symptom, input_medications):
    # Encode the input symptom and medications
    input_symptom_encoded = tokenizer.texts_to_sequences([input_symptom])
    input_medications_encoded = tokenizer.texts_to_sequences([input_medications])
    # Pad the encoded sequences to a fixed length
    input_symptom_padded = tf.keras.preprocessing.sequence.pad_sequences(input_symptom_encoded, maxlen=maxlen)
    input_medications_padded = tf.keras.preprocessing.sequence.pad_sequences(input_medications_encoded, maxlen=maxlen)
    # Predict the reason
    prediction = model.predict([input_symptom_padded, input_medications_padded])
    # Decode the predicted reason
    predicted_reason = dt["reason"].unique()[prediction.argmax()]
    return predicted_reason

Epoch 1/50
25/25 [==============================] - 4s 118ms/step - loss: 6.6254 - accuracy: 0.0013
Epoch 2/50
25/25 [==============================] - 3s 115ms/step - loss: 6.6090 - accuracy: 0.0226
Epoch 3/50
25/25 [==============================] - 3s 116ms/step - loss: 6.5721 - accuracy: 0.0138
Epoch 4/50
25/25 [==============================] - 3s 117ms/step - loss: 6.4638 - accuracy: 0.0138
Epoch 5/50
25/25 [==============================] - 3s 117ms/step - loss: 6.4443 - accuracy: 0.0075
Epoch 6/50
25/25 [==============================] - 3s 117ms/step - loss: 6.1423 - accuracy: 0.0113
Epoch 7/50
25/25 [==============================] - 3s 117ms/step - loss: 5.8743 - accuracy: 0.0188
Epoch 8/50
25/25 [==============================] - 3s 117ms/step - loss: 5.5753 - accuracy: 0.0289
Epoch 9/50
25/25 [==============================] - 3s 116ms/step - loss: 5.2642 - accuracy: 0.0377
Epoch 10/50
25/25 [==============================] - 3s 114ms/step - loss: 4.9325 - accuracy: 0.0616

In [34]:
input_symptom = "Anxiety and nervousness, Depressive or psychotic symptoms, Sharp chest pain"
input_medications = "Lorazepam, Paroxetine (Paxil), Venlafaxine (Effexor)"
predicted_reason = generate_text(model, input_symptom, input_medications)
print(predicted_reason)

1/1 [==============================] - 0s 248ms/step
 diphtheria toxoid vaccine / tetanus toxoid vaccine, 


In [58]:
# Create a tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=436)

# Fit the tokenizer on the symptom column
tokenizer.fit_on_texts(dt["Symptom"])

# Define the input sequence length
maxlen = 100

# Encode the symptom column
X_symptom = tokenizer.texts_to_sequences(dt["Symptom"])

# Pad the encoded sequences to a fixed length
X_symptom = tf.keras.preprocessing.sequence.pad_sequences(X_symptom, maxlen=maxlen)

# Define the model
input_symptom = tf.keras.layers.Input(shape=(maxlen,))
x = tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128)(input_symptom)
x = tf.keras.layers.LSTM(128)(x)
output_reason = tf.keras.layers.Dense(len(tokenizer.word_index)+1, activation='softmax', name='reason')(x)
output_medications = tf.keras.layers.Dense(len(tokenizer.word_index)+1, activation='softmax', name='medications')(x)
model = tf.keras.Model(inputs=input_symptom, outputs=[output_reason, output_medications])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Encode the reason and commonMedications columns
Y_reason = tokenizer.texts_to_matrix(dt["reason"], mode='binary')
Y_medications = tokenizer.texts_to_matrix(dt["commonMedications"], mode='binary')

# Train the model
model.fit(X_symptom, {"reason": Y_reason, "medications": Y_medications}, epochs=100, batch_size=35)

# Generate text using the trained model
def generate_text(model, input_symptom):
    # Tokenize the input symptom
    input_symptom_sequence = tokenizer.texts_to_sequences([input_symptom])
    # Pad the input sequence
    input_symptom_padded = tf.keras.preprocessing.sequence.pad_sequences(input_symptom_sequence, maxlen=100, padding='post', truncating='post')
    # Generate the prediction
    prediction = model.predict(input_symptom_padded)
    # Decode the predicted reason and commonMedications
    predicted_reason = tokenizer.sequences_to_texts([[tf.argmax(prediction[0][i]).numpy() for i in range(len(prediction[0]))]])[0]
    predicted_medications = tokenizer.sequences_to_texts([[tf.argmax(prediction[1][i]).numpy() for i in range(len(prediction[1]))]])[0]
    return tuple(predicted_reason), tuple(predicted_medications)


Epoch 1/100
23/23 [==============================] - 3s 64ms/step - loss: 35.9373 - reason_loss: 31.2637 - medications_loss: 4.6736 - reason_accuracy: 0.1859 - medications_accuracy: 0.7550
Epoch 2/100
23/23 [==============================] - 1s 64ms/step - loss: 20.0520 - reason_loss: 19.3734 - medications_loss: 0.6786 - reason_accuracy: 0.2148 - medications_accuracy: 0.8606
Epoch 3/100
23/23 [==============================] - 1s 65ms/step - loss: 19.9878 - reason_loss: 19.5364 - medications_loss: 0.4514 - reason_accuracy: 0.2148 - medications_accuracy: 0.8606
Epoch 4/100
23/23 [==============================] - 1s 65ms/step - loss: 20.7143 - reason_loss: 20.2808 - medications_loss: 0.4334 - reason_accuracy: 0.2148 - medications_accuracy: 0.8606
Epoch 5/100
23/23 [==============================] - 1s 64ms/step - loss: 21.5621 - reason_loss: 21.1274 - medications_loss: 0.4347 - reason_accuracy: 0.2148 - medications_accuracy: 0.8606
Epoch 6/100
23/23 [==============================] - 1s

In [60]:
input_symptom = "Anxiety and nervousness, Depressive or psychotic symptoms, Sharp chest pain"
predicted_reason, predicted_medications = list(generate_text(model, input_symptom))
print("Predicted Reason:", predicted_reason)
print("Predicted Medications:", predicted_medications)

1/1 [==============================] - 0s 21ms/step
Predicted Reason: ('o', 'f')
Predicted Medications: ("'",)
